# Mapa de Rotas por Equipe usando OSRM + Folium

Este notebook:
- Lê um arquivo `.parquet` de resultados do V3 (por dia).
- Filtra por `dt_ref` e por `equipe`.
- Ordena as OS pela hora de chegada (`dth_chegada_estimada`).
- Usa **OSRM** para montar a rota real (base → OS1 → OS2 → ... → base), perna a perna.
- Plota tudo no **Folium**:
  - Rota rodoviária (quando o OSRM consegue calcular).
  - Linha reta como fallback quando alguma perna falha.
  - Marcadores numerados na ordem de atendimento, com popups detalhados.

In [ ]:
import sys
import os
from pathlib import Path

import pandas as pd
import folium
import requests


In [ ]:
# Caminho do arquivo parquet gerado pelo V3
# Exemplo típico: "results_v3/atribuicoes_2023-01-01.parquet"
PARQUET_PATH = Path("/Rotas-Inteligentes/results_v3/atribuicoes_2023-01-01.parquet")

# Data (dt_ref) que você quer visualizar
# Use None para pegar todas as datas do arquivo
DIA_REF = "2023-01-01"  # ou None

# Equipe que você quer visualizar no mapa
EQUIPE_ALVO = "PVLSN81"

# Caminho do HTML de saída para salvar o mapa
OUTPUT_HTML = (
    Path(f"mapa_osrm_{EQUIPE_ALVO}_{DIA_REF}.html")
    if DIA_REF
    else Path(f"mapa_osrm_{EQUIPE_ALVO}.html")
)

# ==========================
# URL do OSRM
# ==========================
# 1) Tenta importar da sua aplicação (v2.config.OSRM_URL)
OSRM_URL = None
try:
    # Ajuste o path abaixo se necessário para apontar para a raiz do projeto
    sys.path.append(os.getcwd())
    from v2 import config as v2_config

    OSRM_URL = v2_config.OSRM_URL.rstrip("/")
    print(f"OSRM_URL carregada de v2.config: {OSRM_URL}")
except Exception:
    # 2) Se não conseguir importar, defina manualmente aqui:
    OSRM_URL = "http://localhost:5000"  # ajuste conforme a sua instância do OSRM
    print(f"Usando OSRM_URL manual: {OSRM_URL}")


In [ ]:
# Carrega o parquet
df = pd.read_parquet(PARQUET_PATH, engine="pyarrow")
df = pd.DataFrame(df)  # garante que é um DataFrame do pandas

# Verifica colunas necessárias
required_cols = {
    "tipo_serv",
    "numos",
    "datasol",
    "dataven",
    "datater_trab",
    "TD",
    "TE",
    "equipe",
    "dthaps_ini",
    "dthaps_fim_ajustado",
    "inicio_turno",
    "fim_turno",
    "dth_chegada_estimada",
    "dth_final_estimada",
    "fim_turno_estimado",
    "eta_source",
    "base_lon",
    "base_lat",
    "chegada_base",
    "latitude",
    "longitude",
    "dt_ref",
}

missing_cols = required_cols - set(df.columns)
if missing_cols:
    raise ValueError(f"Colunas faltando no parquet: {missing_cols}")

# Normaliza dt_ref para date
df["dt_ref"] = pd.to_datetime(df["dt_ref"], errors="coerce").dt.date

# Filtra por dia, se especificado
if DIA_REF is not None:
    dia_ref_date = pd.to_datetime(DIA_REF).date()
    df = df[df["dt_ref"] == dia_ref_date].copy()

# Filtra por equipe
df_eq = df[df["equipe"] == EQUIPE_ALVO].copy()

if df_eq.empty:
    raise ValueError(
        f"Nenhuma linha encontrada para equipe '{EQUIPE_ALVO}' no dia '{DIA_REF}'."
    )

# Ordena pela hora de chegada estimada (ordem de atendimento)
df_eq["dth_chegada_estimada"] = pd.to_datetime(
    df_eq["dth_chegada_estimada"], errors="coerce"
)
df_eq = df_eq.sort_values("dth_chegada_estimada")
df_eq.reset_index(drop=True, inplace=True)

df_eq.head(10)


In [ ]:
def osrm_route_polyline_segmentado(
    coords, osrm_url: str, profile: str = "driving", geometries: str = "geojson"
):
    """
    Monta a rota completa baseando-se em chamadas OSRM /route por PERNA:
      (p0 -> p1), (p1 -> p2), ..., (pn-1 -> pn)

    coords: lista de tuplas (lon, lat) na ordem:
        base -> OS1 -> OS2 -> ... -> última OS -> base

    Retorna:
        lista de (lat, lon) representando TODO o trajeto concatenado.
    """
    if not coords or len(coords) < 2:
        return []

    rota_latlon = []

    for i in range(len(coords) - 1):
        start = coords[i]
        end = coords[i + 1]
        lon1, lat1 = start
        lon2, lat2 = end

        # Para cada perna, tentar usar OSRM /route
        coords_str = f"{lon1},{lat1};{lon2},{lat2}"
        url = f"{osrm_url}/route/v1/{profile}/{coords_str}"
        params = {
            "overview": "full",
            "geometries": geometries,  # "geojson"
        }

        try:
            resp = requests.get(url, params=params, timeout=10)
            resp.raise_for_status()
            data = resp.json()

            if "routes" in data and data["routes"]:
                geometry = data["routes"][0].get("geometry")
                if geometries == "geojson" and geometry and "coordinates" in geometry:
                    coords_osrm = geometry["coordinates"]  # [[lon, lat], ...]
                    latlon_seg = [(c[1], c[0]) for c in coords_osrm]

                    # Evitar repetir o primeiro ponto de cada segmento (menos o primeiro)
                    if rota_latlon:
                        latlon_seg = latlon_seg[1:]

                    rota_latlon.extend(latlon_seg)
                    continue  # próxima perna se deu certo
        except Exception as e:
            # Qualquer falha cai no fallback de linha reta
            print(f"[OSRM] Falha na perna {i}: {e}")

        # Fallback: linha reta entre os dois pontos
        seg_fallback = [(lat1, lon1), (lat2, lon2)]
        # Evitar repetir o primeiro ponto se já existe rota
        if rota_latlon:
            seg_fallback = seg_fallback[1:]
        rota_latlon.extend(seg_fallback)

    return rota_latlon


In [ ]:
def construir_mapa_equipe_osrm(
    df_equipe: pd.DataFrame, equipe: str, osrm_url: str
) -> folium.Map:
    """Constrói um mapa Folium para a equipe.

    - Base (base_lat/base_lon)
    - Serviços em ordem de chegada (dth_chegada_estimada)
    - Rota real via OSRM ligando base -> serviços -> base (perna a perna)
    - Ordem de atendimento numerada (1, 2, 3, ...)
    """
    # Ordenar explicitamente pela hora de chegada
    df_equipe = df_equipe.copy()
    df_equipe["dth_chegada_estimada"] = pd.to_datetime(
        df_equipe["dth_chegada_estimada"], errors="coerce"
    )
    df_equipe = df_equipe.sort_values("dth_chegada_estimada").reset_index(drop=True)

    # Pega base (assumindo que é igual em todas as linhas da equipe)
    base_lon = float(df_equipe["base_lon"].iloc[0])
    base_lat = float(df_equipe["base_lat"].iloc[0])

    # Lista de pontos da rota: base -> serviços (na ordem) -> base
    coords_osrm = []
    coords_osrm.append((base_lon, base_lat))  # base inicial (lon, lat)

    # Serviços na ordem de chegada
    pontos_servicos = []  # (lat, lon) para colocar marcadores
    for _, row in df_equipe.iterrows():
        lat = float(row["latitude"])
        lon = float(row["longitude"])
        coords_osrm.append((lon, lat))
        pontos_servicos.append((lat, lon))

    # Base final
    coords_osrm.append((base_lon, base_lat))

    # Cria o mapa centralizado na base
    m = folium.Map(location=[base_lat, base_lon], zoom_start=12, tiles="OpenStreetMap")

    # Marca a base
    folium.CircleMarker(
        location=[base_lat, base_lon],
        radius=7,
        color="blue",
        fill=True,
        fill_color="blue",
        popup=f"Base equipe {equipe}",
        tooltip=f"Base {equipe}",
    ).add_to(m)

    # Chama o OSRM (segmentado) para pegar a geometria da rota
    try:
        linha_rota = osrm_route_polyline_segmentado(coords_osrm, osrm_url)
        if linha_rota:
            folium.PolyLine(
                locations=linha_rota,
                color="orange",
                weight=4,
                opacity=0.8,
                tooltip=f"Rota equipe {equipe} (OSRM)",
            ).add_to(m)
        else:
            folium.Marker(
                location=[base_lat, base_lon],
                popup="OSRM não retornou geometria; rota não desenhada.",
                icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa"),
            ).add_to(m)
    except Exception as e:
        folium.Marker(
            location=[base_lat, base_lon],
            popup=f"Erro ao obter rota OSRM segmentada: {e}",
            icon=folium.Icon(color="red", icon="exclamation-triangle", prefix="fa"),
        ).add_to(m)

    # Adiciona marcadores para cada OS, numerados na ordem de atendimento
    for ordem, (_, row) in enumerate(df_equipe.iterrows(), start=1):
        lat = float(row["latitude"])
        lon = float(row["longitude"])
        numos = row["numos"]
        tipo = row["tipo_serv"]
        chegada = row["dth_chegada_estimada"]
        fim = row["dth_final_estimada"]
        te = row["TE"]
        td = row["TD"]

        popup_html = (
            f"<b>Ordem:</b> {ordem}<br>"
            f"<b>OS:</b> {numos}<br>"
            f"<b>Tipo:</b> {tipo}<br>"
            f"<b>Chegada:</b> {chegada}<br>"
            f"<b>Fim:</b> {fim}<br>"
            f"<b>TE:</b> {te} min<br>"
            f"<b>TD:</b> {td} min<br>"
        )

        cor = "red" if tipo == "técnico" else "green"

        folium.CircleMarker(
            location=[lat, lon],
            radius=6,
            color=cor,
            fill=True,
            fill_color=cor,
            popup=folium.Popup(popup_html, max_width=300),
            tooltip=f"{ordem} - OS {numos} ({tipo})",
        ).add_to(m)

        # Rótulo com o número da ordem (pequeno texto ao lado)
        folium.map.Marker(
            [lat, lon],
            icon=folium.DivIcon(
                html=f'<div style="font-size: 10pt; color: black;"><b>{ordem}</b></div>'
            ),
        ).add_to(m)

    return m


In [ ]:
mapa = construir_mapa_equipe_osrm(df_eq, EQUIPE_ALVO, OSRM_URL)
mapa
